In [5]:
# Importing

from langchain_core.embeddings import Embeddings
from infinispan_vector import Infinispan

In [6]:
# Demo with a real embedding
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L12-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)


In [7]:
# Creating a langchain_core.VectorStore
# the default store name `allminilm` will be used

ispn_allminilm = Infinispan.from_texts(embedding=hf,
                                 configuration={"lambda.content": lambda item: item["sentence"], "cache_name": "allminilm", "entity_name" : "allmini_vec"})

In [8]:
# Creating an Infinispan cache to store vectors

cache_def = '''
{
  "distributed-cache": {
    "owners": "2",
    "mode": "SYNC",
    "statistics": true,
    "encoding": {
      "media-type": "application/x-protostream"
    },
    "indexing": {
      "enabled": true,
      "storage": "filesystem",
      "startup-mode": "AUTO",
      "indexing-mode": "AUTO",
      "indexed-entities": [
        "allmini_vec"
      ]
    }
  }
}
'''
ispn_allminilm.req_cache_post("allminilm",cache_def)

<Response [200]>

In [9]:
import json
schema = '''
/**
 * @Indexed
 */
message allmini_vec {
/**
 * @Vector(dimension=384)
 */
repeated float floatVector = 1;
optional string texture = 2;
optional string sentence = 3;
}
'''
output = ispn_allminilm.req_schema_post("allmini_vec.proto",schema)
print(output.text)
assert output.status_code == 200
assert json.loads(output.text)["error"] == None

{"name":"allmini_vec.proto","error":null}


In [10]:
# Adding some data
texts =    [{"_key": 1, "_type": "allmini_vec", "texture" : "matt", "sentence" : "Volleyball is a team sport in which two teams of six players are separated by a net. Each team tries to score points by grounding a ball on the other team's court under organized rules."},
                {"_key": 2, "_type": "allmini_vec", "texture": "glossy", "sentence": "An earthquake – also called a quake, tremor, or temblor – is the shaking of the surface of Earth resulting from a sudden release of energy in the lithosphere that creates seismic waves."},
                {"_key": 3, "_type": "allmini_vec", "texture": "silk", "sentence": "An octopus is a soft-bodied, eight-limbed mollusc of the order Octopoda. The order consists of some 300 species and is grouped within the class Cephalopoda with squids, cuttlefish, and nautiloids."},
                {"_key": 4, "_type": "allmini_vec", "texture": "matt", "sentence": "Basketball is a team sport in which two teams, most commonly of five players each, opposing one another on a rectangular court, compete with the primary objective of shooting a basketball (approximately 9.4 inches (24 cm) in diameter) through the defender's hoop."},
                {"_key": 5, "_type": "allmini_vec", "texture": "raw", "sentence": "Physics is the natural science of matter, involving the study of matter,[a] its fundamental constituents, its motion and behavior through space and time, and the related entities of energy and force."},
                                 ]
res = ispn_allminilm.add_texts(texts)
res

['1', '2', '3', '4', '5']

In [11]:
ispn_allminilm.similarity_search("I like swimming in the sea",2)

[Document(page_content='An octopus is a soft-bodied, eight-limbed mollusc of the order Octopoda. The order consists of some 300 species and is grouped within the class Cephalopoda with squids, cuttlefish, and nautiloids.'),
 Document(page_content="Volleyball is a team sport in which two teams of six players are separated by a net. Each team tries to score points by grounding a ball on the other team's court under organized rules.")]

In [12]:
ispn_allminilm.similarity_search("I don't like tennis, I prefer playing in a team",2)

[Document(page_content="Volleyball is a team sport in which two teams of six players are separated by a net. Each team tries to score points by grounding a ball on the other team's court under organized rules."),
 Document(page_content="Basketball is a team sport in which two teams, most commonly of five players each, opposing one another on a rectangular court, compete with the primary objective of shooting a basketball (approximately 9.4 inches (24 cm) in diameter) through the defender's hoop.")]

In [13]:
# Clean up
ispn_allminilm.req_cache_delete("allminilm")
ispn_allminilm.req_schema_delete("allmini_vec.proto")

<Response [204]>